In [25]:
#import libraries 
import pandas as pd
import requests
import time
import pymysql as pml

In [26]:
def get_anime(number_of_animes):
#Create a list to store the requests
    response = []
#Call api in a loop because has 500 as limit for anime titles requests
    for i in range (0, number_of_animes, 500):
        url = "https://api.myanimelist.net/v2/anime/ranking?fields=start_date, end_date, mean, genres, media_type, status, num_episodes, rating&limit=500&offset={}".format(i)
        headers = {'X-MAL-CLIENT-ID' : '3988e33ad8c7197c95ab720d9092f47c'}
        response.append(requests.get(url, headers = headers).json())
        time.sleep(1) #give it a second before starting the for loop
    return response

In [27]:
def get_anime_details(response, df):
#cleaning up the data from response
    for items in response:
        for anime in items['data']:
            anime_id = anime['node']['id']
            title = anime['node']['title']
            if 'genres' in anime['node']:
                genre = anime['node']['genres'][0]['name']
            else:
                genre = ' '
            mean_score = anime['node']['mean']
            media_type = anime['node']['media_type']
            num_episodes = anime['node']['num_episodes']
            picture = anime['node']['main_picture']['medium']
            if 'rating' in anime['node']:
                rating = anime['node']['rating']
            else:
                rating = ' '
            status = anime['node']['status']
            if 'start_date' in anime['node']:
                start_date = anime['node']['start_date']
            else:
                start_date = ' '
            if 'end_date' in anime['node']:
                end_date = anime['node']['end_date']
            else:
                end_date = ' '

#Save data in pandas dataframe
    df = df.append({ 'anime_id': anime_id, 
                      'title': title, 
                      'genre': genre, 
                      'mean_score': mean_score, 
                      'media_type': media_type, 
                      'num_episodes': num_episodes, 
                      'picture': picture,
                      'rating': rating, 
                      'status': status, 
                      'start_date': start_date, 
                      'end_date': end_date}, ignore_index= True)
    return df

In [28]:
def connect_to_db(host_name, username, password, dbname, port):
    try:  
        conn = pml.connect(host= host_name, user = username, password = password, port = port,database = dbname )
    
    except pml.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn 

In [29]:
def create_table(curr):
    create_table_command = (""" CREATE TABLE IF NOT EXISTS animes (
                            anime_id INT(255) PRIMARY KEY,
                            title VARCHAR(120) NOT NULL,
                            genre VARCHAR(120),
                            mean_score FLOAT(3,2),
                            media_type VARCHAR(15) NOT NULL,
                            num_episodes INT(6),
                            picture VARCHAR(200) NOT NULL,
                            rating VARCHAR(15) NOT NULL,
                            status VARCHAR(60) NOT NULL,
                            start_date DATE NOT NULL,
                            end_date DATE
                        )""")
    
    curr.execute(create_table_command)
    return None

In [30]:
def check_if_anime_exists(curr, anime_id):
    query = ("""SELECT anime_id FROM animes WHERE anime_id = %s """)
    curr.execute(query, (anime_id,))

    return curr.fetchone() is not None

In [31]:
def update_row(curr, title, genre, mean_score, media_type, num_episodes, 
                      picture, rating, status, start_date,end_date, anime_id):
    query = (""" UPDATE animes
                SET title = %s,             
                genre = %s,
                mean_score = %s,
                media_type = %s,
                num_episodes = %s,
                picture = %s,
                rating = %s,
                status = %s,
                start_date = %s,
                end_date = %s
                WHERE anime_id = %s;
                """)

    vars_to_update = (title, genre, mean_score, media_type, num_episodes, 
                      picture, rating, status, start_date,end_date, anime_id)
    curr.execute(query, vars_to_update)
    return None

In [32]:
def update_db(curr, df):    
    temp_df = pd.DataFrame(columns = df.columns)

    for i, row in df.iterrows():
        if check_if_anime_exists(curr, row['anime_id']): #if the video already exists the it will update
            update_row(curr, row['title'], row['genre'], row['mean_score'], row['media_type'], 
                       row['num_episodes'], row['picture'], row['rating'], row['status'], 
                       row['start_date'], row['end_date'], row['anime_id'])
        else: # The video doesn't exists so it will add it to a temp df and append it using append_from_df_to_db
            temp_df = temp_df.append(row)
            
    return temp_df

In [33]:
def insert_into_table(curr, anime_id, title, genre, mean_score, media_type,
    num_episodes, picture, rating, status, start_date, end_date):

    insert = (""" INSERT INTO animes( anime_id,
                title,             
                genre,
                mean_score,
                media_type,
                num_episodes,
                picture,
                rating,
                status,
                start_date,
                end_date)
                VALUES(%s, %s, %s, %s ,%s ,%s ,%s ,%s ,%s ,%s, %s)
                """)
    row_to_insert = (anime_id, title, genre, mean_score, media_type,
    num_episodes, picture, rating, status, start_date, end_date)
    
    curr.execute(insert, row_to_insert)
    
    return None

In [34]:
def apprend_from_df_to_db(curr, df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['anime_id'],  row['title'], row['genre'], row['mean_score'], row['media_type'], 
                       row['num_episodes'], row['picture'], row['rating'], row['status'], 
                       row['start_date'], row['end_date'])
    return None

In [ ]:
#main

#Build a Dataframe
df = pd.DataFrame(columns = ['anime_id', 'title', 'genre', 'mean_score', 'media_type', 'num_episodes', 'picture', 'rating', 'status', 'start_date', 'end_date'])

#get 8000 anime titles from api and load into the dataframe
response = get_anime(8000)
df = get_anime_details(response, df)

#database credentials
host_name = 'personalproject.crpcbnfr4nbk.us-east-1.rds.amazonaws.com'
username = 'admin'
password = 'ad5c7455'
dbname = 'personal_projects'
port = 3306
conn = None

#establish a connection to db
conn = connect_to_db(host_name, username, password, dbname, port)
curr = conn.cursor()

# Create table
create_table(curr)

In [42]:
#update data for existing animes
new_anime_df = update_db(curr, df)
conn.commit()

In [43]:
#insert new videos into db table
apprend_from_df_to_db(curr, new_anime_df)
conn.commit()

In [44]:
#view data in db table
curr.execute("SELECT * FROM animes")
print(curr.fetchall())

((1, 'Cowboy Bebop', 'Action', 8.76, 'tv', 26, 'https://api-cdn.myanimelist.net/images/anime/4/19644.jpg', 'r', 'finished_airing', datetime.date(1998, 4, 3), datetime.date(1999, 4, 24)), (5, 'Cowboy Bebop: Tengoku no Tobira', 'Action', 8.38, 'movie', 1, 'https://api-cdn.myanimelist.net/images/anime/1439/93480.jpg', 'r', 'finished_airing', datetime.date(2001, 9, 1), datetime.date(2001, 9, 1)), (6, 'Trigun', 'Action', 8.22, 'tv', 26, 'https://api-cdn.myanimelist.net/images/anime/7/20310.jpg', 'pg_13', 'finished_airing', datetime.date(1998, 4, 1), datetime.date(1998, 9, 30)), (7, 'Witch Hunter Robin', 'Action', 7.26, 'tv', 26, 'https://api-cdn.myanimelist.net/images/anime/1796/91065.jpg', 'pg_13', 'finished_airing', datetime.date(2002, 7, 2), datetime.date(2002, 12, 24)), (15, 'Eyeshield 21', 'Shounen', 7.92, 'tv', 145, 'https://api-cdn.myanimelist.net/images/anime/12/66961.jpg', 'pg_13', 'finished_airing', datetime.date(2005, 4, 6), datetime.date(2008, 3, 19)), (16, 'Hachimitsu to Clover